In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np

In [2]:
tittles=[]
years=[]
certificates=[]
runtimes=[]
genres=[]
ratings=[]
metascore=[]
overview = []
directors=[]
actors=[]
listFilm=[]
count =0

In [3]:
for i in range (0,5000,50):
  url  = "https://www.imdb.com/search/title/?title_type=feature&year=1945-01-01,2021-12-31&sort=num_votes,desc&start="
  response = requests.get(url, headers={"Accept-Language": "en-US"})
  soup = BeautifulSoup(response.text, "html.parser")
  count=count+1
  movies= soup.find_all('div', class_='lister-item mode-advanced')

  for movie in movies:
    aFilm=[]
    tittle = movie.h3.find('a', href=True).text
    year = movie.h3.find('span', class_ = 'lister-item-year text-muted unbold').text
    year = year.split(" ")
    if len(year)>1:
      year=year[1]
      #tittle=tittle+year[0]
      #print(tittle,year)
    else:
      year=year[0]
    aFilm.append(tittle)
    aFilm.append(year)

    rating=float(movie.strong.text)
    aFilm.append(rating)

    nv = movie.find_all('span', attrs = {'name':'nv'})
    vote=nv[0].text
    aFilm.append(vote)
    
    meta = movie.find("div", class_="ratings-metascore")
    if meta is not None:
      metascore = meta.span.get_text()
    else:
      meta = "Unknown"
    aFilm.append(metascore)
    
    gross= nv[1].text if len(nv) > 1 else '-'
    aFilm.append(gross) 
  
    ct=movie.find_all('span', class_="certificate")
    certificate=ct[0].text
    aFilm.append(certificate)

    rt=movie.find_all('span', class_="runtime")
    runtime=rt[0].text
    aFilm.append(runtime)

    gr=movie.find_all('span', class_="genre")
    genre=gr[0].text
    genre=genre[1:]
    genre=genre[:-12]
    aFilm.append(genre)
    
    overview = movie.find_all("p", class_="text-muted")[1].text
    aFilm.append(overview)
    
    star=[]
    filminfo1=movie.find_all('p',class_='')
    f2=filminfo1[0]
    director=f2.find_all('a')[0]
    director = director.text
    
    for a in f2.find_all('a')[1:]:
      star.append(a.text)
  
    aFilm.append(star)

    if len(star) >4:
      director=star[0]+"; "+director
      del star[0]
    aFilm.append(director)
    listFilm.append(aFilm)

In [4]:
df = pd.DataFrame(listFilm, columns =['Title', 'Year', 'Rating','Vote','Metascore','Budget','Certificate','Runtime','Genre','Overview','Artist','Director'])
df.head()

,Title,Year,Rating,Vote,Metascore,Budget,Certificate,Runtime,Genre,Overview,Artist,Director
0,The Shawshank Redemption,(1994),9.3,"2,750,801",82,$28.34M,R,142 min,Drama,"\nOver the course of several years, two convic...","[Tim Robbins, Morgan Freeman, Bob Gunton, Will...",Frank Darabont
1,The Dark Knight,(2008),9.0,"2,723,579",84,$534.86M,PG-13,152 min,"Action, Crime, Drama",\nWhen the menace known as the Joker wreaks ha...,"[Christian Bale, Heath Ledger, Aaron Eckhart, ...",Christopher Nolan
2,Inception,(2010),8.8,"2,417,423",74,$292.58M,PG-13,148 min,"Action, Adventure, Sci-Fi",\nA thief who steals corporate secrets through...,"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elli...",Christopher Nolan
3,Fight Club,(1999),8.8,"2,190,483",66,$37.03M,R,139 min,Drama,\nAn insomniac office worker and a devil-may-c...,"[Brad Pitt, Edward Norton, Meat Loaf, Zach Gre...",David Fincher
4,Forrest Gump,(1994),8.8,"2,139,972",82,$330.25M,PG-13,142 min,"Drama, Romance","\nThe presidencies of Kennedy and Johnson, the...","[Tom Hanks, Robin Wright, Gary Sinise, Sally F...",Robert Zemeckis


In [5]:
df.shape

(5000, 12)

In [7]:
df.to_csv("Movies.csv")

In [3]:
df=pd.read_csv("Movies.csv")

In [4]:
df.isna().sum()

Unnamed: 0     0
Title          0
Year           0
Rating         0
Vote           0
Metascore      0
Budget         0
Certificate    0
Runtime        0
Genre          0
Overview       0
Artist         0
Director       0
dtype: int64

In [5]:
df.shape

(5000, 13)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   5000 non-null   int64  
 1   Title        5000 non-null   object 
 2   Year         5000 non-null   object 
 3   Rating       5000 non-null   float64
 4   Vote         5000 non-null   object 
 5   Metascore    5000 non-null   int64  
 6   Budget       5000 non-null   object 
 7   Certificate  5000 non-null   object 
 8   Runtime      5000 non-null   object 
 9   Genre        5000 non-null   object 
 10  Overview     5000 non-null   object 
 11  Artist       5000 non-null   object 
 12  Director     5000 non-null   object 
dtypes: float64(1), int64(2), object(10)
memory usage: 507.9+ KB


In [7]:
df.describe()

,Unnamed: 0,Rating,Metascore
count,5000.000000,5000.000000,5000.000000
mean,2499.500000,8.526000,78.280000
std,1443.520003,0.316139,10.970217
min,0.000000,7.900000,59.000000
25%,1249.750000,8.300000,69.000000
50%,2499.500000,8.500000,78.000000
75%,3749.250000,8.700000,87.000000
max,4999.000000,9.300000,100.000000


In [10]:
df.iloc[50]

Unnamed: 0                                                    50
Title                                   The Shawshank Redemption
Year                                                      (1994)
Rating                                                       9.3
Vote                                                   2,750,801
Metascore                                                     82
Budget                                                   $28.34M
Certificate                                                    R
Runtime                                                  142 min
Genre                                                      Drama
Overview       \nOver the course of several years, two convic...
Artist         ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...
Director                                          Frank Darabont
Name: 50, dtype: object